# Model

<a href="https://huggingface.co/google/gemma-2-2b-it">google/gemma-2-2b-it</a>

This notebook should just contain code examples and a base working training model, once a basic working model is built we will freeze any work on this model, it will become my local reference.

# Raw Material Locations, anything that has not had significant updates will not be includes

### Stage 1 - Easy things to grab

- Intel Github https://github.com/intel
  - Items to be processed
  - https://github.com/intel/ipex-llm
  - https://github.com/intel/intel-extension-for-pytorch
  - https://github.com/intel/ai
  - So the great thing about having all these in github is we can simply clone all this data, and since its written in markdown it will be easily converted into a Q/A format by another LLM.
- Arc B580 press release slides
  - https://download.intel.com/newsroom/2024/client-computing/Intel-Arc-B580-B570-Media-Deck.pdf
  - https://cdrdv2-public.intel.com/839907/Intel%20Arc%20B-Series%20Graphics%20Quick%20Reference%20Guide%20V1.1.pdf
- My own instructions
  - https://github.com/phillipscarroll/ml-xpu/blob/main/native_win11_intel_arc_xpu_pytorch-amp.md

# Human Created Questions and Answers



In [ ]:
# Using an upscaled FP32 precision

# pip install accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-it",
    device_map="auto",
)

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=32)
print(tokenizer.decode(outputs[0]))


# CSV Example

In [ ]:
question,answer
"What is a tensor?","A tensor is a multi-dimensional array used in machine learning and deep learning."
"How does oneAPI optimize matrix multiplication?","oneAPI uses Intel’s DPC++ to enable vectorization and memory-efficient execution."
"What is mixed precision training?","Mixed precision training uses both FP32 and FP16 to reduce memory usage and improve training speed."

# CSV Ingest Example

In [ ]:
import pandas as pd

df = pd.read_csv("dataset.csv")
print(df.head())  # Check data

# Code Examples From Gemma 2

In [ ]:
# Using 16-bit BF16 precision

# pip install accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=32)
print(tokenizer.decode(outputs[0]))


In [ ]:
# Using 8-bit precision

# pip install bitsandbytes accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-it",
    quantization_config=quantization_config,
)

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=32)
print(tokenizer.decode(outputs[0]))

In [ ]:
# Using 4-bit precision

# pip install bitsandbytes accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True)

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-it",
    quantization_config=quantization_config,
)

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=32)
print(tokenizer.decode(outputs[0]))